In [2]:
from typing import Dict, Optional, List
import numpy as np

class HMMState:
    def __init__(self, mean: List[np.ndarray], covariance: List[np.ndarray], transition: Dict["HMMState", float], label: Optional[int] = None, parent: Optional["HMMState"] = None):
        self.mean = mean
        """n_gaussians of mean vectors."""
        self.covariance = covariance
        """n_gaussians of diagonal of covariance matrix."""
        self.transition = transition
        """Transition probability to other HMMState instances."""
        self.label = label
        """The digit associated with the state. `None` if the state is the first state."""
        self.parent = parent
        """The state is the first state if the `parent` is `None`."""

    @classmethod
    def root(cls):
        """Creates a root HMMState with default parameters."""
        return cls(mean=[], covariance=[], transition={}, label=None)

    def __hash__(self) -> int:
        """Enables HMMState instances to be used as dictionary keys or in sets."""
        return id(self)

    def add_transition(self, state: "HMMState", probability: float):
        """Adds or updates a transition probability to another state."""
        self.transition[state] = probability

    def get_transition_prob(self, state: "HMMState") -> float:
        """Retrieves the transition probability to another state, if defined."""
        return self.transition.get(state, 0.0)
    
    def get_emission_prob(self, observation: np.ndarray) -> float:
        """Calculate the emission probability of an observation for this state,
        considering the state might represent multiple Gaussians."""
        total_prob = 0.0
        n_gaussians = len(self.mean)
        
        for i in range(n_gaussians):
            mean = self.mean[i]
            covariance = self.covariance[i]
            k = mean.shape[0]
            covariance_det = np.prod(covariance)
            covariance_inv = 1 / covariance
            diff = observation - mean
            exponent = -0.5 * np.sum((diff ** 2) * covariance_inv)
            coefficient = 1 / np.sqrt((2 * np.pi) ** k * covariance_det)
            total_prob += coefficient * np.exp(exponent)
        
        # Assuming equal weight for each Gaussian component
        return total_prob / n_gaussians if n_gaussians > 0 else 0

# Example of usage
if __name__ == "__main__":
    # Creating root state
    root_state = HMMState.root()
    
    # Creating another state with example data
    mean_example = [np.array([0.0, 1.0])]
    covariance_example = [np.array([1.0, 1.0])]
    state_example = HMMState(mean=mean_example, covariance=covariance_example, transition={}, label=1)
    
    # Adding a transition from root to example state
    root_state.add_transition(state_example, 0.5)

    print(root_state.get_transition_prob(state_example))  # Example of getting a transition probability


0.5


In [19]:
from typing import List, Dict,Tuple
import numpy as np
import os


class HMM:
    def __init__(self):
        self.states: List[HMMState] = []
        self.observations: List[np.ndarray] = []
        self.transitions: List[List[float]] = []
        self.state_index: Dict[HMMState, int] = {}
        self.initial_probabilities: List[float] = []  # Probability of starting in each state

    def add_state(self, state: HMMState, initial_probability: float = None):
        """Adds a state to the HMM."""
        self.states.append(state)
        index = len(self.states) - 1
        self.state_index[state] = index
        # Expand transitions matrix
        for row in self.transitions:
            row.append(0.0)
        self.transitions.append([0.0] * len(self.states))
        # Set initial probability (if provided)
        if initial_probability is not None:
            self.initial_probabilities.append(initial_probability)
        else:
            # Default to uniform distribution if not specified
            self.initial_probabilities.append(1.0 / len(self.states))

    def set_observations(self, observations: List[np.ndarray]):
        """Sets the sequence of observations for the HMM."""
        self.observations = observations

    def viterbi(self) -> Tuple[List[int], float]:
        """Finds the most probable sequence of states given the observations using the Viterbi algorithm."""
        num_states = len(self.states)
        num_observations = len(self.observations)
        if num_observations == 0:
            return [], 0.0  # No observations, return empty path and zero probability

        # Initialize DP tables
        dp = np.zeros((num_states, num_observations))
        backpointer = np.zeros((num_states, num_observations), dtype=int)

        # Initialization step
        for s in range(num_states):
            dp[s, 0] = self.initial_probabilities[s] * self.states[s].emission_probability(self.observations[0])

        # Recursion step
        for t in range(1, num_observations):
            for s in range(num_states):
                transition_probabilities = [dp[prev_state, t-1] * self.transitions[prev_state][s] for prev_state in range(num_states)]
                max_transition_prob = max(transition_probabilities)
                dp[s, t] = max_transition_prob * self.states[s].emission_probability(self.observations[t])
                backpointer[s, t] = transition_probabilities.index(max_transition_prob)

        # Termination step
        best_path_prob = max(dp[:, num_observations-1])
        best_last_state = np.argmax(dp[:, num_observations-1])

        # Backtrack to find best path
        best_path = [best_last_state]
        for t in range(num_observations-1, 0, -1):
            best_last_state = backpointer[best_last_state, t]
            best_path.insert(0, best_last_state)

        return best_path, best_path_prob
    def initial_segmentation(self, templates, num_segments):
    
        segmented_templates = []

        for template in templates:
        # Determine the size of each segment
            num_observations = len(template)
            segment_size = num_observations // num_segments
            extra = num_observations % num_segments

            segments = []
            start_idx = 0

            for _ in range(num_segments):
            # Adjust segment size to distribute remaining observations
                end_idx = start_idx + segment_size + (1 if extra > 0 else 0)
            # Decrease extra count until it's distributed
                extra -= 1 if extra > 0 else 0

            # Extract the segment and add to the list
                segment = template[start_idx:end_idx]
                segments.append(segment)

                start_idx = end_idx

            segmented_templates.append(segments)

        return segmented_templates

    
    def load_features_and_train(training_folder):
        # Placeholder for loading features and training HMM for each digit
        print(f"Loading features from {training_folder}...")
        # Here we would load the .npy files and train each digit's HMM
        # For now, we will just print out what would normally happen.
        for digit in range(10):
            for i in range(1, 11):  # 10 feature files per digit
                feature_path = os.path.join(training_folder, f"{digit}-{i}.npy")
                print(f"Training HMM for digit {digit} with features from {feature_path}...")
                # Here we would load the features using numpy.load(feature_path)
                # And then train the HMM for this digit using these features


    
    def train(self, sequences, n_iterations=100, convergence_tol=1e-6):
        """
        Train the HMM using the Expectation-Maximization algorithm.
        
        Args:
            sequences (List[List[np.ndarray]]): List of all observation sequences.
            n_iterations (int): Number of iterations to run the EM algorithm.
            convergence_tol (float): The convergence tolerance. Training stops when the change in log-likelihood is less than this value.
        """
        previous_log_likelihood = None
        
        for iteration in range(n_iterations):
            # E Step
            all_alphas = [self._forward(seq) for seq in sequences]
            all_betas = [self._backward(seq) for seq in sequences]
            
            # Compute log-likelihood
            log_likelihood = sum(
                np.log(np.sum(alphas[-1])) for alphas in all_alphas
            )
            
            # Check for convergence
            if previous_log_likelihood is not None and abs(log_likelihood - previous_log_likelihood) < convergence_tol:
                print(f"Model converged after {iteration} iterations.")
                break
            previous_log_likelihood = log_likelihood
            
            # M Step
            self._update_states(sequences, all_alphas, all_betas)
            self._update_transitions(sequences, all_alphas, all_betas)
            
            # Optionally print the log-likelihood every few iterations to monitor the training progress
            if iteration % 10 == 0:
                print(f"Iteration {iteration}: Log Likelihood = {log_likelihood}")
        
        print("Training complete.")

    def _forward(self, obs_seq):
        """
        Forward algorithm for calculating the probability of the observation sequence.

        Args:
            obs_seq (List[np.ndarray]): Observation sequence.

        Returns:
            np.ndarray: Matrix of probabilities (states x observations).
        """
        n_states = len(self.states)
        n_observations = len(obs_seq)
        alpha = np.zeros((n_states, n_observations))

        # Initialization
        for s in range(n_states):
            alpha[s, 0] = self.states[s].get_emission_prob(obs_seq[0]) * (1.0 / n_states)

        # Induction
        for t in range(1, n_observations):
            for s in range(n_states):
                alpha[s, t] = self.states[s].get_emission_prob(obs_seq[t]) * sum(
                    alpha[prev_s, t - 1] * self.states[prev_s].get_transition_prob(self.states[s])
                    for prev_s in range(n_states)
                )
        
        return alpha

    def _backward(self, obs_seq):
        """
        Backward algorithm for calculating the probability of the future observations given current state.

        Args:
            obs_seq (List[np.ndarray]): Observation sequence.

        Returns:
            np.ndarray: Matrix of probabilities (states x observations).
        """
        n_states = len(self.states)
        n_observations = len(obs_seq)
        beta = np.zeros((n_states, n_observations))

        # Initialization
        beta[:, n_observations - 1] = 1  # Set all to 1 for the final probabilities

        # Induction
        for t in range(n_observations - 2, -1, -1):
            for s in range(n_states):
                beta[s, t] = sum(
                    self.states[s].get_transition_prob(self.states[next_s]) *
                    self.states[next_s].get_emission_prob(obs_seq[t + 1]) * beta[next_s, t + 1]
                    for next_s in range(n_states)
                )
        
        return beta

    def _update_states(self, sequences, all_alphas, all_betas):
        """
        Update the emission probabilities (means and covariances) for each state.
        
        Args:
            sequences (List[List[np.ndarray]]): List of all observation sequences.
            all_alphas (List[np.ndarray]): List of alpha matrices from the forward algorithm.
            all_betas (List[np.ndarray]): List of beta matrices from the backward algorithm.
        """
        for s in range(len(self.states)):
            # Accumulators for means and variances
            weighted_sum = 0
            weighted_square_sum = 0
            normalizer = 0

            for seq_idx, obs_seq in enumerate(sequences):
                alphas = all_alphas[seq_idx]
                betas = all_betas[seq_idx]
                for t in range(len(obs_seq)):
                    # Calculate the weight for this observation at time t
                    weight = alphas[s, t] * betas[s, t]
                    # Update the accumulators
                    weighted_sum += weight * obs_seq[t]
                    weighted_square_sum += weight * np.outer(obs_seq[t], obs_seq[t])
                    normalizer += weight

            # Update the means and variances for the state
            if normalizer > 0:
                new_mean = weighted_sum / normalizer
                new_covariance = weighted_square_sum / normalizer - np.outer(new_mean, new_mean)
                self.states[s].mean = [new_mean]  # Assuming a single Gaussian for simplicity
                self.states[s].covariance = [np.diag(new_covariance)]  # Assuming diagonal covariance for simplicity

    def _update_transitions(self, sequences, all_alphas, all_betas):
        """
        Update the transition probabilities between states.
        
        Args:
            sequences (List[List[np.ndarray]]): List of all observation sequences.
            all_alphas (List[np.ndarray]): List of alpha matrices from the forward algorithm.
            all_betas (List[np.ndarray]): List of beta matrices from the backward algorithm.
        """
        for s in range(len(self.states)):
            for next_s in range(len(self.states)):
                numerator = 0
                denominator = 0

                for seq_idx, obs_seq in enumerate(sequences):
                    alphas = all_alphas[seq_idx]
                    betas = all_betas[seq_idx]
                    for t in range(len(obs_seq) - 1):
                        numerator += (alphas[s, t] *
                                    self.states[s].get_transition_prob(self.states[next_s]) *
                                    self.states[next_s].get_emission_prob(obs_seq[t + 1]) *
                                    betas[next_s, t + 1])

                        denominator += alphas[s, t] * betas[s, t]

                # Update the transition probability from state s to state next_s
                if denominator > 0:
                    self.states[s].transition[self.states[next_s]] = numerator / denominator

    
    def decode(self, sequence):
        """
        Decode a sequence of observations and return the most probable state sequence using the Viterbi algorithm.
        Args:
            sequence (List[np.ndarray]): An observation sequence.
        Returns:
            List[int]: The most likely state sequence.
        """
        return self.viterbi(sequence)[0]

    def evaluate(self, sequences, labels):
        """
        Evaluate the HMM on a test set.
        Args:
            sequences (List[List[np.ndarray]]): A list of observation sequences.
            labels (List[List[int]]): The true state sequences for each observation sequence.
        Returns:
            float, float: The sentence accuracy and the word accuracy.
        """
        correct_sentences = 0
        correct_words = 0
        total_words = 0

        for obs_seq, true_states in zip(sequences, labels):
            predicted_states = self.decode(obs_seq)

            if predicted_states == true_states:
                correct_sentences += 1

            correct_words += sum(p == t for p, t in zip(predicted_states, true_states))
            total_words += len(true_states)

        sentence_accuracy = correct_sentences / len(sequences)
        word_accuracy = correct_words / total_words

        return sentence_accuracy, word_accuracy
def hmm_load_features(data_dir):
    #创建一个空列表来存储样本对象
    samples = []

    #*循环读取数据并创建样本对象，每个样本对象包括特征数据和标签
    for file_name in os.listdir(data_dir):
        if file_name.endswith('.npy'):
            # 解析文件名，获取标签信息
            parts = file_name.split('-')
            if len(parts) == 2 and parts[1].endswith('.npy'):
                label = int(parts[0])
                
                # 加载特征数据
                features = np.load(os.path.join(data_dir, file_name))

                # 创建样本对象并添加到列表中
                sample = {'label': label, 'features': features}
                samples.append(sample)

    return samples
def filter_samples_by_label(samples, label=1):
    """
    Filters the list of sample dictionaries to include only those with a specific label.

    :param samples: List of dictionaries, where each dictionary contains 'label' and 'features' keys.
    :param label: The label to filter by (default is 1).
    :return: A filtered list of dictionaries.
    """
    return [sample["features"] for sample in samples if sample['label'] == label]
    # Assuming the training folder path is correct
training_folder_path = '../training'  # This path will need to be updated to the actual path
hmm=HMM()
#hmm.load_features_and_train(training_folder_path)
data=hmm_load_features(training_folder_path)
#print(data)
#for digit in range(10):
        #for i in range(1, 11):  # 10 feature files per digit
            #feature_path = os.path.join(training_folder_path, f"{digit}-{i}.npy")
            #print(f"Training HMM for digit {digit} with features from {feature_path}...")
for i in range(10):
    i_thdata=filter_samples_by_label(data, label=i)
    print(f'the features for digit {i} is {i_thdata}')



the features for digit 0 is [array([[-1.32525131, -0.99562569,  0.89454906, ..., -0.66694999,
        -0.51300302,  1.18688686],
       [-0.97012792, -1.37641652,  1.22428018, ...,  0.50863305,
        -1.62939205,  1.82605719],
       [-0.89731153, -1.65849   ,  1.18766476, ...,  0.00870973,
        -0.86951147,  1.71887036],
       ...,
       [-1.37190858, -0.59784399,  0.56582384, ..., -0.17132877,
        -0.47327292,  0.67167253],
       [-1.3919091 , -0.60294529,  0.69113072, ...,  0.01340227,
        -0.81733797,  0.51877819],
       [-1.49957289, -0.72604153,  0.62635545, ...,  0.18777759,
        -0.37962377,  0.24810123]]), array([[ 0.15008617, -2.36730339,  1.65577399, ...,  0.48452049,
        -0.92574404,  0.65044752],
       [ 0.21633839, -2.2649048 ,  2.07606437, ...,  0.20940006,
        -0.78318265,  1.04616966],
       [ 0.39421503, -1.28766847,  2.23317194, ...,  0.43007705,
        -0.60072796,  0.75469489],
       ...,
       [-1.2305068 , -1.01853114,  0.36078497

In [ ]:
class TelephoneNumberFSM:
    def __init__(self):
        self.current_state = 'start'
        self.states=['start','area_1','area_2','area_3','number_1','number_2','number_3','number_4','end']
        # Initialize transitions with area code skip option
        self.transitions = {
            'start': {},
            'area_1': {},
            'area_2': {},
            'area_3': {},
            'number_1': {},
            'number_2': {},
            'number_3': {},
            'number_4': {},
            'end': {}
        }
        # Define valid digits for transitions
        self.valid_digits = [str(i) for i in range(10)]  # '0' to '9'
        self.first_digit_options = [str(i) for i in range(2, 10)]  # '2' to '9'
        # Populate transitions using loops
        self.populate_transitions()

    def populate_transitions(self):
        # Start state transitions
        for digit in self.first_digit_options:
            self.transitions['start'][digit] = 'area_1'
        self.transitions['start']['skip'] = 'number_1'  # Skip to number if no area code
        
        # Area code transitions
        for state in ['area_1', 'area_2', 'area_3']:
            for digit in self.valid_digits:
                next_state = 'area_3' if state == 'area_2' else 'number_1' if state == 'area_3' else 'area_2'
                self.transitions[state][digit] = next_state
        
        # Number transitions
        current_state = 'number_1'
        for i in range(1, 5):  # number_1 to number_4
            for digit in self.valid_digits:
                next_state = 'end' if i == 4 else f'number_{i+1}'
                self.transitions[f'number_{i}'][digit] = next_state

    def transition(self, input):
        if input in self.transitions[self.current_state]:
            self.current_state = self.transitions[self.current_state][input]
        else:
            raise ValueError(f"Invalid transition from {self.current_state} with input {input}")

    def is_valid_number(self):
        return self.current_state == 'end'

# Example usage
fsm = TelephoneNumberFSM()
try:
    for digit in "2851234":  # Example number without area code
        fsm.transition(digit)
    print(fsm.is_valid_number())  # Check if it's a valid telephone number
except ValueError as e:
    print(e)


False


In [ ]:
digit_states = {i: HMMState.root() for i in range(10)}

for i in range(10):
    for j in range(10):
        if i != j: 
            digit_states[i].add_transition(digit_states[j], 0.1)

silence_state = HMMState.root()
for state in digit_states.values():
    state.add_transition(silence_state, 0.05) 
    silence_state.add_transition(state, 0.05)

hmm_model = HMM()
for state in digit_states.values():
    hmm_model.add_state(state)
hmm_model.add_state(silence_state)

